# Flagging days/periods with Snow

The following code will take December 2017 data from the King Street Pilot, compare it to baseline values, and will include a flag corresponding to whether or not it snowed in the respective time period. 

In [1]:
# import relevant modules
import configparser
from psycopg2 import connect
import psycopg2.sql as pg
import pandas.io.sql as pandasql
import datetime

# connect to database
CONFIG = configparser.ConfigParser()
CONFIG.read(r'C:\Users\alouis2\Documents\Python Scripts\db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [29]:
# run query
snow_periods = {datetime.date(2017,12,11):['AM Peak', 'PM Peak', 'Evening'], 
                datetime.date(2017,12,12):['AM Peak', 'Midday'],
                datetime.date(2017,12,13):['AM Peak', 'Evening'],
                datetime.date(2017,12,15):['Midday', 'PM Peak'],
                datetime.date(2017,12,18):['AM Peak'],
                datetime.date(2017,12,21):['Evening'],
                datetime.date(2017,12,22):['AM Peak', 'Midday', 'PM Peak'],
                datetime.date(2017,12,29):['AM Peak', 'Midday']}


def formatter(snow_periods):
    string = ''' dt = '%s' '''
    for i in range(len(snow_periods)-1):
        string = string + '''or dt = '%s' '''
    string = string % tuple([key for key in snow_periods])
    return  '''select pilot.dt, pilot.street, pilot.direction, pilot.period, 
                pilot.tt as pilot, baseline.tt as baseline 
                from king_pilot.dash_daily_dev as pilot,  king_pilot.dash_baseline_dev as baseline 
                where (%s)
                and category = 'Pilot' and  pilot.day_type = 'Weekday' and pilot.street = baseline.street 
                and pilot.direction = baseline.direction and pilot.day_type = baseline.day_type 
                and pilot.period = baseline.period order by dt, street, direction, period''' % string




def flagged(snow_periods):
    string = formatter(snow_periods)
    sql = pg.SQL(string)
    snow = pandasql.read_sql(sql, con)
    change = [snow.pilot.values[i]-snow.baseline.values[i] for i in range(len(snow))]
    snow['change'] = change
    flag = [0 for i in range(len(snow))]
    snow['snow_flag'] = flag 
    for key in snow_periods:
        for i in snow_periods[key]:
            for j in range(len(snow)):
                if snow.dt.values[j] == key and snow.period.values[j] == i:
                    snow.snow_flag.values[j] += 1        
    return snow

